In [ ]:
!pip install -q transformers==4.51.3 datasets evaluate seqeval

import torch, transformers, datasets, evaluate, os, json, numpy as np
print("Torch 版本:", torch.__version__)
print("Transformers 版本:", transformers.__version__)
print("GPU:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "CPU")


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 3.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.4 MB 110.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 37.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 16.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2024.12.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12=

In [ ]:
from google.colab import files
uploaded = files.upload()   # 选中你的 bio_labeled_output_alcon_2022_esg_text_blocks.jsonl
DATA_PATH = list(uploaded.keys())[0]
print("已上传:", DATA_PATH)

Saving bio_labeled_output_alcon_2022_esg_text_blocks.jsonl to bio_labeled_output_alcon_2022_esg_text_blocks.jsonl
已上传: bio_labeled_output_alcon_2022_esg_text_blocks.jsonl


In [ ]:
def load_jsonl(path):
    with open(path, encoding="utf-8") as f:
        return [json.loads(l) for l in f]

raw_data = load_jsonl(DATA_PATH)
print("样本数:", len(raw_data), "  示例:", raw_data[0])

from datasets import Dataset
dataset = Dataset.from_list(raw_data).train_test_split(test_size=0.1, seed=42)


样本数: 371   示例: {'tokens': ['202', '##2', 'S', '##OC', '##IA', '##L', 'I', '##MP', '##AC', '##T', 'AND', '##SU', '##ST', '##A', '##IN', '##AB', '##IL', '##IT', '##Y', 'R', '##EP', '##OR', '##T'], 'labels': ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], 'text': '2022 SOCIAL IMPACT ANDSUSTAINABILITY REPORT'}


In [ ]:
unique_labels = sorted({l for d in raw_data for l in d["labels"]})
label2id = {l:i for i,l in enumerate(unique_labels)}
id2label = {i:l for l,i in label2id.items()}
num_labels = len(label2id)
print("标签集合:", unique_labels)

标签集合: ['B-B-ENV_ENC', 'B-B-ENV_GHG', 'B-B-ENV_WAC', 'B-B-ENV_WAG_TWG', 'B-B-GOV_ASS_ASR', 'B-B-GOV_BOC_WOB', 'B-B-GOV_ETB_ACD', 'B-B-GOV_MAD_WMT', 'B-B-SOC_GED_CEG', 'B-B-SOC_GED_NHG', 'B-B-SOC_OHS_FAT', 'B-B-SOC_OHS_HCI', 'I-B-ENV_ENC', 'I-B-ENV_GHG', 'I-B-ENV_WAC', 'I-B-ENV_WAG_TWG', 'I-B-GOV_ASS_ASR', 'I-B-GOV_BOC_WOB', 'I-B-GOV_ETB_ACD', 'I-B-GOV_MAD_WMT', 'I-B-SOC_GED_CEG', 'I-B-SOC_GED_NHG', 'I-B-SOC_OHS_FAT', 'I-B-SOC_OHS_HCI', 'O']


In [ ]:
from transformers import AutoTokenizer
tok = AutoTokenizer.from_pretrained("bert-base-cased")

IGNORE = -100
def tok_align(ex):
    enc = tok(ex["tokens"],
              is_split_into_words=True,
              truncation=True,
              padding="max_length",
              max_length=128)
    word_ids = enc.word_ids()
    labels = []
    prev = None
    for w in word_ids:
        if w is None:
            labels.append(IGNORE)
        elif w != prev:
            labels.append(label2id[ex["labels"][w]])
        else:
            labels.append(label2id[ex["labels"][w]])
        prev = w
    enc["labels"] = labels
    return enc

ds_tok = dataset.map(tok_align, batched=False, remove_columns=dataset["train"].column_names)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

Map:   0%|          | 0/333 [00:00<?, ? examples/s]

Map:   0%|          | 0/38 [00:00<?, ? examples/s]

In [ ]:
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer
model = AutoModelForTokenClassification.from_pretrained(
    "bert-base-cased",
    num_labels=num_labels,
    id2label=id2label,
    label2id=label2id
)


args = TrainingArguments(
    output_dir="./ner_output",
    do_eval=True,                      # ✅ 替代 evaluation_strategy="epoch"
    logging_steps=10,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    save_total_limit=2,
)



Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
import numpy as np
import evaluate

metric = evaluate.load("seqeval")

def compute_metrics(p):
    predictions, labels = p
    preds = np.argmax(predictions, axis=2)

    true_predictions = []
    true_labels = []

    for pred, label in zip(preds, labels):
        pred_labels = []
        true_labels_seq = []
        for p, l in zip(pred, label):
            if l != -100:
                pred_labels.append(id2label[p])
                true_labels_seq.append(id2label[l])
        true_predictions.append(pred_labels)
        true_labels.append(true_labels_seq)

    return metric.compute(predictions=true_predictions, references=true_labels)


In [ ]:
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=ds_tok["train"],
    eval_dataset=ds_tok["test"],
    tokenizer=tok,
    compute_metrics=compute_metrics,
)

trainer.train()


<ipython-input-13-f4004de12202>:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: jiayuzhou20 (jiayuzhou20-nus) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss
10,0.824700
20,0.061100
30,0.061000
40,0.048000
50,0.030800
60,0.062400
70,0.036700
80,0.048200
90,0.073900
100,0.030600


TrainOutput(global_step=126, training_loss=0.11169688001511589, metrics={'train_runtime': 22.23, 'train_samples_per_second': 44.939, 'train_steps_per_second': 5.668, 'total_flos': 65272435027200.0, 'train_loss': 0.11169688001511589, 'epoch': 3.0})

In [ ]:
trainer.evaluate()

/usr/local/lib/python3.11/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.11/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
Trainer is attempting to log a value of "{'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 1}" of type <class 'dict'> for key "eval/B-ENV_WAG_TWG" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 1}" of type <class 'dict'> for key "eval/B-GOV_ASS_ASR" a

{'eval_loss': 0.07164744287729263,
 'eval_B-ENV_WAG_TWG': {'precision': 0.0,
  'recall': 0.0,
  'f1': 0.0,
  'number': 1},
 'eval_B-GOV_ASS_ASR': {'precision': 0.0,
  'recall': 0.0,
  'f1': 0.0,
  'number': 1},
 'eval_B-GOV_ETB_ACD': {'precision': 0.0,
  'recall': 0.0,
  'f1': 0.0,
  'number': 1},
 'eval_B-GOV_MAD_WMT': {'precision': 0.0,
  'recall': 0.0,
  'f1': 0.0,
  'number': 1},
 'eval_B-SOC_GED_CEG': {'precision': 0.0,
  'recall': 0.0,
  'f1': 0.0,
  'number': 3},
 'eval_B-SOC_OHS_HCI': {'precision': 0.0,
  'recall': 0.0,
  'f1': 0.0,
  'number': 1},
 'eval_overall_precision': 0.0,
 'eval_overall_recall': 0.0,
 'eval_overall_f1': 0.0,
 'eval_overall_accuracy': 0.9906692406692407,
 'eval_runtime': 0.1661,
 'eval_samples_per_second': 228.776,
 'eval_steps_per_second': 30.102,
 'epoch': 3.0}